In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv("e.csv")

In [3]:
data.head()

,text,label
0,Just got back from seeing @GaryDelaney in Burs...,joy
1,Oh dear an evening of absolute hilarity I don'...,joy
2,Been waiting all week for this game ❤️❤️❤️ #ch...,joy
3,"@gardiner_love : Thank you so much, Gloria! Yo...",joy
4,I feel so blessed to work with the family that...,joy


In [4]:
data.shape

(3613, 2)

In [5]:
df=data.dropna()
df.shape

(3613, 2)

In [6]:
df.label.unique()

array(['joy', 'fear', 'anger', 'sadness'], dtype=object)

In [7]:
df.text.value_counts()

i love the word fret so much and im in heaven                                                                                                   2
@TehShockwave turn that grumpy frown upside-down\n\nYou did something next to impossible today                                                  2
@UltimateBoxer My heart because you left me for so long again\n\n*slight pout but it turned to a smile*\n\nheheh just kidding, no I'm fine-     2
@ArcadianLuthier -- taking out his feelings on Kei unfairly. His lips form a frown as he tries to walk away.                                    2
@cburt43 turn that frown upside down                                                                                                            2
                                                                                                                                               ..
Rollover car crash on warren st, traffic is horrid #Boston                                                                  

In [8]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [9]:
# remove whitespaces
df['text']=df['text'].str.strip()
# lowercase the text
df['text'] = df['text'].str.lower()
#remove punctuation
punc = string.punctuation
table = str.maketrans('','',punc)
df['text']=df['text'].apply(lambda x: x.translate(table))
# tokenizing each message
df['text']=df.apply(lambda x: x['text'].split(' '),axis=1)
# removing stopwords
df['text'] = df.apply(lambda x: [word for word in x['text'] if word not in stopwords.words('english')],axis=1)
# stemming
ps = PorterStemmer()
df['text']= df.apply(lambda x: [ps.stem(word) for word in x['text']],axis=1)
# remove single letter words
df['text'] = df.apply(lambda x: ' '.join([word for word in x['text'] if len(word)>1]),axis=1)


In [10]:
X = np.array(df["text"])
y = np.array(df["label"])

In [11]:
X[0]

'got back see garydelaney burslem amaz face still hurt laugh much hilari'

In [12]:
y[0:10]

array(['joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy', 'joy',
       'joy'], dtype=object)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv_X = cv.fit_transform(X) # Fit the Data

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cv_X, y, test_size=0.2, random_state=42)

In [15]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train,y_train)

RandomForestClassifier()

In [16]:
predict = clf.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predict)*100)

82.57261410788381


In [18]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,predict))

[[123  25   2   4]
 [  0 204   4  15]
 [  1  22 159   3]
 [ 12  30   8 111]]


In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predict))

              precision    recall  f1-score   support

       anger       0.90      0.80      0.85       154
        fear       0.73      0.91      0.81       223
         joy       0.92      0.86      0.89       185
     sadness       0.83      0.69      0.76       161

    accuracy                           0.83       723
   macro avg       0.85      0.82      0.83       723
weighted avg       0.84      0.83      0.83       723



In [20]:
import joblib
joblib.dump(cv,'lr_cv.pkl')
joblib.dump(clf,'lr.pkl')

['lr.pkl']